In [80]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/train-data/call_train_data.parquet
/kaggle/input/transformation/scikitlearn/default/1/scaler_x.pkl
/kaggle/input/transformation/scikitlearn/default/1/power_transformer_x.pkl
/kaggle/input/put-train-data/put_train
/kaggle/input/nk-iv-prediction/test_data.parquet
/kaggle/input/nk-iv-prediction/sample_submission.csv
/kaggle/input/nk-iv-prediction/train_data.parquet
/kaggle/input/call-test/calltest.parquet
/kaggle/input/transform/scikitlearn/default/1/scaler_x.pkl


In [81]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')
import pickle
from sklearn.preprocessing import PowerTransformer
from sklearn.preprocessing import RobustScaler
from sklearn.preprocessing import MinMaxScaler

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import DataLoader, TensorDataset, random_split

import numpy as np
from sklearn.metrics import mean_squared_error

In [82]:
train = pd.read_parquet('/kaggle/input/train-data/call_train_data.parquet')

train.head(5)

train.shape

(4078811, 45)

In [83]:
X_remove = ['X0', 'X4', 'X5', 'X6', 'X7', 'X9', 'X11', 'X13', 'X15', 'X16', 'X20', 'X22', 'X23', 'X25', 'X28', 'X29', 'X30', 'X34', 'X38', 'X39', 'X41']

train.drop(columns = X_remove, inplace = True)

train.shape

(4078811, 24)

In [210]:
from sklearn.model_selection import train_test_split

sampled_df = train.sample(frac=1, random_state=42)

X = sampled_df.drop('iv', axis=1) # Features

Y = sampled_df['iv']               # Target

x_train, x_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, random_state=42)


x_train.shape,y_train.shape,x_test.shape,y_test.shape

((3263048, 23), (3263048,), (815763, 23), (815763,))

In [211]:
scaler_call= MinMaxScaler()
x_train = scaler_call.fit_transform(x_train)
x_test = scaler_call.transform(x_test)

In [212]:
x_train.max()
x_train.min()
x_test.max()
x_test.min()

0.0

In [267]:
from sklearn.metrics import mean_squared_error
from xgboost import XGBRegressor
xg_call = XGBRegressor(
    n_estimators=25,
    learning_rate=0.1,
    max_depth = 1,
    random_state=42,
    n_jobs=-1,
    tree_method='gpu_hist'
)

xg_call.fit(x_train, y_train)

# Step 5: Predict on test set
y_pred = xg_call.predict(x_test)

# Step 6: Calculate MSE
mse = mean_squared_error(y_test,y_pred)
print(f"Test MSE: {mse:.4f}")


Test MSE: 0.0036


In [320]:
from lightgbm import LGBMRegressor
from sklearn.metrics import mean_squared_error

# ✅ Basic LightGBM model
lbm = LGBMRegressor(
    n_estimators=600,
    learning_rate=0.001,
    max_depth=15,
    random_state=42,
    device='gpu',           # ✅ Use 'gpu' for GPU training (optional)
    boosting_type='gbdt'    # Gradient Boosting Decision Tree (default)
)

# Fit the model
lbm.fit(x_train, y_train)

# Predict
y_pred = model.predict(x_test)

# Evaluate
mse = mean_squared_error(y_test, y_pred)
print(f"LightGBM Test MSE: {mse:.4f}")


[LightGBM] [Info] This is the GPU trainer!!
[LightGBM] [Info] Total Bins 5636
[LightGBM] [Info] Number of data points in the train set: 3263048, number of used features: 23
[LightGBM] [Info] Using GPU Device: Tesla P100-PCIE-16GB, Vendor: NVIDIA Corporation
[LightGBM] [Info] Compiling OpenCL Kernel with 256 bins...
[LightGBM] [Info] GPU programs have been built
[LightGBM] [Info] Size of histogram bin entry: 8
[LightGBM] [Info] 20 dense feature groups (62.24 MB) transferred to GPU in 0.077937 secs. 1 sparse feature groups
[LightGBM] [Info] Start training from score 0.229579
LightGBM Test MSE: 0.0075


In [321]:
df_call.isna().sum()

x_input = df_call.drop(columns = ['iv']+ X_remove)


x_input = MinMaxScaler().fit_transform(x_input)

y_input = df_call['iv']

y_pred = lbm.predict(x_input)

# Step 6: Calculate MSE
mse = mean_squared_error(y_input,y_pred)
print(f"Test MSE: {mse:.4f}")

Test MSE: 0.0033


In [215]:
x_train.shape

(3263048, 23)

In [268]:
df_call.isna().sum()

x_input = df_call.drop(columns = ['iv']+ X_remove)


x_input = MinMaxScaler().fit_transform(x_input)

y_input = df_call['iv']

y_pred = xg_call.predict(x_input)

# Step 6: Calculate MSE
mse = mean_squared_error(y_input,y_pred)
print(f"Test MSE: {mse:.4f}")




Test MSE: 0.0037


In [317]:
with open('/kaggle/working/xg_call.pkl','wb') as f:
    pickle.dump(xg_call,f)

In [50]:
!nvidia-smi


Thu Jun  5 07:47:09 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 560.35.03              Driver Version: 560.35.03      CUDA Version: 12.6     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla P100-PCIE-16GB           Off |   00000000:00:04.0 Off |                    0 |
| N/A   40C    P0             35W /  250W |     307MiB /  16384MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [95]:
put_train = pd.read_parquet('/kaggle/input/put-train-data')

In [96]:
put_train.shape

(4636840, 45)

In [97]:
col = 'iv'
Q1 = put_train[col].quantile(0.25)
Q3 = put_train[col].quantile(0.75)
IQR = Q3 - Q1

lower_bound = Q1 - 1.5 * IQR
upper_bound = Q3 + 1.5 * IQR

# Boolean mask for outliers
outliers_mask = (put_train[col] < lower_bound) | (put_train[col] > upper_bound)




# Calculate percentage
percent_outliers = outliers_mask.mean() * 100

print(f"Percentage of outliers in '{col}': {percent_outliers:.2f}%")

Percentage of outliers in 'iv': 8.40%


In [98]:
put_train_no_outliers = put_train[(put_train['iv'] >= lower_bound) & (put_train['iv'] <= upper_bound)]
print(f"Original data size: {put_train.shape[0]}")
print(f"Data size after removing outliers: {put_train_no_outliers.shape[0]}")


Original data size: 4636840
Data size after removing outliers: 4247491


In [99]:
put_train = put_train_no_outliers
put_train.shape

(4247491, 45)

In [100]:
X_remove = ['X0', 'X4', 'X5', 'X6', 'X7', 'X9', 'X11', 'X13', 'X15', 'X16', 'X20', 'X22', 'X23', 'X25', 'X28', 'X29', 'X30', 'X34', 'X38', 'X39', 'X41']

put_train.drop(columns = X_remove, inplace = True)

put_train.shape

(4247491, 24)

In [269]:
from sklearn.model_selection import train_test_split

sampled_df = put_train.sample(frac=1, random_state=42)

X = sampled_df.drop('iv', axis=1) # Features

Y = sampled_df['iv']               # Target

put_x_train, put_x_test, put_y_train, put_y_test = train_test_split(X, Y, test_size=0.2, random_state=42)


put_x_train.shape,put_y_train.shape,put_x_test.shape,put_y_test.shape

((3397992, 23), (3397992,), (849499, 23), (849499,))

In [270]:
scaler_put = MinMaxScaler()
put_x_train = scaler_put.fit_transform(put_x_train)
put_x_test = scaler_put.transform(put_x_test)

In [103]:
print(put_x_train.max())
print(put_x_test.max())
put_x_test.min()

1.0000000000000002
1.0000000000000002


0.0

In [283]:

from sklearn.metrics import mean_squared_error
from xgboost import XGBRegressor
xg_put = XGBRegressor(
    n_estimators=25,
    learning_rate=0.1,
    max_depth=1,
    random_state=42,
    n_jobs=-1
)

xg_put.fit(put_x_train, put_y_train)

# Step 5: Predict on test set
put_y_pred = xg_put.predict(put_x_test)

# Step 6: Calculate MSE
mse = mean_squared_error(put_y_test, put_y_pred)
print(f"Test MSE: {mse:.4f}")


Test MSE: 0.0148


In [284]:
x_put_test = df.drop(columns = ['iv']+X_remove)
x_put_test = MinMaxScaler().fit_transform(x_put_test)

y_input_put = df['iv']

y_pred = xg_put.predict(x_put_test)

# Step 6: Calculate MSE
mse = mean_squared_error(y_input_put,y_pred)
print(f"Test MSE: {mse:.4f}")

Test MSE: 0.0029


In [318]:
with open('/kaggle/working/xg_put.pkl','wb') as f:
    pickle.dump(xg_put,f)

In [191]:
test = pd.read_parquet('/kaggle/input/nk-iv-prediction/test_data.parquet')
test.head(5)

,timestamp,underlying,call_iv_24000,call_iv_24100,call_iv_24200,call_iv_24300,call_iv_24400,call_iv_24500,call_iv_24600,call_iv_24700,...,X32,X33,X34,X35,X36,X37,X38,X39,X40,X41
0,0,24735.9,0.280939,NaN,NaN,NaN,0.242149,NaN,0.232439,NaN,...,0.006587,0.002826,-1.737617e+05,-0.009541,-0.017831,0.000264,2.032521e+06,1.000000e-06,-0.077238,-5.362742e+06
1,1,24766.7,0.270276,NaN,0.258893,NaN,NaN,NaN,0.233548,NaN,...,0.005777,0.004588,-3.195086e+05,-0.024106,-0.004696,-0.000158,-4.529075e+06,-1.619022e+06,-0.956928,4.624907e+06
2,2,24896.9,NaN,0.251731,NaN,NaN,0.214869,0.20458,0.194604,NaN,...,0.000829,-0.034944,1.027525e+06,0.028201,0.032234,0.007687,2.663908e+06,0.000000e+00,-0.116264,-2.669766e+06
3,3,24898.1,0.241888,NaN,0.220505,NaN,0.198602,0.18619,NaN,NaN,...,-0.009323,-0.022969,-4.720074e+06,-0.001513,0.008704,-0.000206,-7.672936e+06,-1.903406e+06,-2.249208,-1.858254e+07
4,4,24906.5,0.235328,NaN,0.222983,0.214126,NaN,NaN,0.192603,NaN,...,-0.018674,-0.007588,-4.051681e+05,-0.136267,0.002425,-0.002200,1.334469e+06,2.548789e+05,1.999104,3.024212e+06


In [192]:
test.drop(columns = ['timestamp'],  inplace = True)
test.head()

,underlying,call_iv_24000,call_iv_24100,call_iv_24200,call_iv_24300,call_iv_24400,call_iv_24500,call_iv_24600,call_iv_24700,call_iv_24800,...,X32,X33,X34,X35,X36,X37,X38,X39,X40,X41
0,24735.9,0.280939,NaN,NaN,NaN,0.242149,NaN,0.232439,NaN,0.222997,...,0.006587,0.002826,-1.737617e+05,-0.009541,-0.017831,0.000264,2.032521e+06,1.000000e-06,-0.077238,-5.362742e+06
1,24766.7,0.270276,NaN,0.258893,NaN,NaN,NaN,0.233548,NaN,NaN,...,0.005777,0.004588,-3.195086e+05,-0.024106,-0.004696,-0.000158,-4.529075e+06,-1.619022e+06,-0.956928,4.624907e+06
2,24896.9,NaN,0.251731,NaN,NaN,0.214869,0.20458,0.194604,NaN,NaN,...,0.000829,-0.034944,1.027525e+06,0.028201,0.032234,0.007687,2.663908e+06,0.000000e+00,-0.116264,-2.669766e+06
3,24898.1,0.241888,NaN,0.220505,NaN,0.198602,0.18619,NaN,NaN,NaN,...,-0.009323,-0.022969,-4.720074e+06,-0.001513,0.008704,-0.000206,-7.672936e+06,-1.903406e+06,-2.249208,-1.858254e+07
4,24906.5,0.235328,NaN,0.222983,0.214126,NaN,NaN,0.192603,NaN,NaN,...,-0.018674,-0.007588,-4.051681e+05,-0.136267,0.002425,-0.002200,1.334469e+06,2.548789e+05,1.999104,3.024212e+06


In [193]:
call = []
for col in test.columns.tolist() :
    if 'call' in col:
        call.append(col)
    else:
        continue
len(call)

put = []
for col in test.columns.tolist() :
    if 'put' in col:
        put.append(col)
    else:
        continue
len(put)  

k_call = []
for col in call:
    col.split('iv')[1]
    k_call.append(col.split('iv')[1].split('_')[1])
len(k_call)

call_test = test.drop(columns = put)
call_test.shape


all_values = []
modified = call_test[call]
for i in range(len(modified)):
    value = modified.iloc[i].tolist()  # 26 values
    all_values.extend(value)
len(all_values)

base_call = call_test.drop(columns = call)
base_call.shape

repeated_indices = np.repeat(base_call.index.values, 26)
expanded_col_call = base_call.loc[repeated_indices].reset_index(drop=True)

expanded_col_call['k'] = k_call * len(base)

expanded_col_call.shape

expanded_col_call['iv'] = all_values

expanded_col_call.head(27)

expanded_col_call['k'] = expanded_col_call['k'].astype(float)

In [195]:
import numpy as np

# Replace None with np.nan
expanded_col_call['iv'] = expanded_col_call['iv'].replace({'None': np.nan})

not_nan_rows = expanded_col_call[expanded_col_call['iv'].notna()]
not_nan_rows.head(10)
df_call = not_nan_rows
nan_rows = expanded_col_call[expanded_col_call['iv'].isna()]
new_df_call = nan_rows
new_df_call.head(10)
print(new_df_call.shape)
df_call.shape


(188055, 45)


(125635, 45)

In [137]:
df.isna().sum()

x_input = df.drop(columns = ['iv']+ X_remove)


x_input = MinMaxScaler().fit_transform(x_input)

y_input = df['iv']

y_pred = xg_put.predict(x_input)

# Step 6: Calculate MSE
mse = mean_squared_error(y_input,y_pred)
print(f"Test MSE: {mse:.4f}")




Test MSE: 0.0088


In [124]:
print("Transformed x_test1 min:", np.min(x_input))
print("Transformed x_test1 max:", np.max(x_input))

Transformed x_test1 min: 0.0
Transformed x_test1 max: 1.0


In [ ]:
torch.save(model.state_dict(), "/kaggle/working/trained_model.pth")


In [126]:
x_input1 = new_df.drop(columns = ['iv']+X_remove)


x_input1 = MinMaxScaler().fit_transform(x_input1)

y_pred = xg_call.predict(x_input1)

new_df['iv'] = y_pred


In [127]:
new_df.head(27)

,underlying,X0,X1,X2,X3,X4,X5,X6,X7,X8,...,X34,X35,X36,X37,X38,X39,X40,X41,k,iv
1,24735.9,-2.377420e+06,-0.066961,-0.064562,-0.068452,23078.869048,117442.708333,-55935.974702,-0.0,-0.001486,...,-173761.718749,-0.009541,-0.017831,0.000264,2.032521e+06,1.000000e-06,-0.077238,-5.362742e+06,24100.0,0.188273
2,24735.9,-2.377420e+06,-0.066961,-0.064562,-0.068452,23078.869048,117442.708333,-55935.974702,-0.0,-0.001486,...,-173761.718749,-0.009541,-0.017831,0.000264,2.032521e+06,1.000000e-06,-0.077238,-5.362742e+06,24200.0,0.182097
3,24735.9,-2.377420e+06,-0.066961,-0.064562,-0.068452,23078.869048,117442.708333,-55935.974702,-0.0,-0.001486,...,-173761.718749,-0.009541,-0.017831,0.000264,2.032521e+06,1.000000e-06,-0.077238,-5.362742e+06,24300.0,0.176345
5,24735.9,-2.377420e+06,-0.066961,-0.064562,-0.068452,23078.869048,117442.708333,-55935.974702,-0.0,-0.001486,...,-173761.718749,-0.009541,-0.017831,0.000264,2.032521e+06,1.000000e-06,-0.077238,-5.362742e+06,24500.0,0.192813
7,24735.9,-2.377420e+06,-0.066961,-0.064562,-0.068452,23078.869048,117442.708333,-55935.974702,-0.0,-0.001486,...,-173761.718749,-0.009541,-0.017831,0.000264,2.032521e+06,1.000000e-06,-0.077238,-5.362742e+06,24700.0,0.172595
10,24735.9,-2.377420e+06,-0.066961,-0.064562,-0.068452,23078.869048,117442.708333,-55935.974702,-0.0,-0.001486,...,-173761.718749,-0.009541,-0.017831,0.000264,2.032521e+06,1.000000e-06,-0.077238,-5.362742e+06,25000.0,0.196792
13,24735.9,-2.377420e+06,-0.066961,-0.064562,-0.068452,23078.869048,117442.708333,-55935.974702,-0.0,-0.001486,...,-173761.718749,-0.009541,-0.017831,0.000264,2.032521e+06,1.000000e-06,-0.077238,-5.362742e+06,25300.0,0.238580
14,24735.9,-2.377420e+06,-0.066961,-0.064562,-0.068452,23078.869048,117442.708333,-55935.974702,-0.0,-0.001486,...,-173761.718749,-0.009541,-0.017831,0.000264,2.032521e+06,1.000000e-06,-0.077238,-5.362742e+06,25400.0,0.238580
16,24735.9,-2.377420e+06,-0.066961,-0.064562,-0.068452,23078.869048,117442.708333,-55935.974702,-0.0,-0.001486,...,-173761.718749,-0.009541,-0.017831,0.000264,2.032521e+06,1.000000e-06,-0.077238,-5.362742e+06,25600.0,0.241828
17,24735.9,-2.377420e+06,-0.066961,-0.064562,-0.068452,23078.869048,117442.708333,-55935.974702,-0.0,-0.001486,...,-173761.718749,-0.009541,-0.017831,0.000264,2.032521e+06,1.000000e-06,-0.077238,-5.362742e+06,25700.0,0.239633


In [128]:
test.head(10)

new_df['k'] = new_df['k'].astype('int32')

new_df['k'].dtype

for i in range(len(new_df)):
    value = new_df['iv'].iloc[i]
    k = new_df['k'].iloc[i]
    col = f'call_iv_{k}'
    cols_to_match = [f'X{i}' for i in range(42)]  # Include 'underlying' if needed

    row_i = new_df.iloc[i]
    
    # Compare each row in test to row_i
    mask = (test[cols_to_match] == row_i[cols_to_match]).all(axis=1)
    
    # Get matching index
    matching_index = test.index[mask]

    # insert
    test.loc[matching_index, col] = value

KeyboardInterrupt: 

In [ ]:
test.isna().sum()

In [ ]:
call_submission = test

In [ ]:
call_submission.head()

In [ ]:
test = pd.read_parquet('/kaggle/input/nk-iv-prediction/test_data.parquet')
test.drop(columns = ['timestamp'],  inplace = True)
test.head()


In [130]:
put_test = test.drop(columns = call)
put_test.shape


all_values_put = []
modified = put_test[put]
for i in range(len(modified)):
    value = modified.iloc[i].tolist()  # 26 values
    all_values_put.extend(value)
len(all_values_put)

base_put = put_test.drop(columns = put)
base_put.shape

repeated_indices = np.repeat(base_put.index.values, 26)
expanded_col_put = base_put.loc[repeated_indices].reset_index(drop=True)

k_put = []
for col in put:
    col.split('iv')[1]
    k_put.append(col.split('iv')[1].split('_')[1])
len(k_put)

expanded_col_put['k'] = k_put * len(base)

expanded_col_put.shape

expanded_col_put['iv'] = all_values_put

expanded_col_put['k'] = expanded_col_put['k'].astype(float)

In [131]:
import numpy as np

# Replace None with np.nan
expanded_col_put['iv'] = expanded_col_put['iv'].replace({'None': np.nan})

not_nan_rows = expanded_col_put[expanded_col_put['iv'].notna()]
not_nan_rows.head(10)
df = not_nan_rows
nan_rows = expanded_col_put[expanded_col_put['iv'].isna()]
new_df = nan_rows
new_df.head(10)
print(new_df.shape)
df.shape

(188449, 45)


(125241, 45)

In [143]:
x_put_test = df.drop(columns = ['iv']+X_remove)
x_put_test = MinMaxScaler().fit_transform(x_put_test)

y_input_put = df['iv']

y_pred = xg_put.predict(x_put_test)

# Step 6: Calculate MSE
mse = mean_squared_error(y_input_put,y_pred)
print(f"Test MSE: {mse:.4f}")

Test MSE: 0.0042


In [ ]:
x_put_input = new_df.drop(columns = ['iv']+X_remove)

pt_x_put_input = PowerTransformer(method='yeo-johnson')
x_put_input = pt_x_put_input.fit_transform(x_put_input)

x_input1_tensor = torch.tensor(x_put_input, dtype=torch.float32)
# 3. Predict using model
model.eval()
with torch.no_grad():
    y_pred_tensor = model(x_input1_tensor)
# 4. Convert predictions back to numpy
y_pred_np = y_pred_tensor.cpu().numpy().flatten()  # Flatten if needed


new_df['iv'] = y_pred_np

In [ ]:
new_df.head()

In [ ]:
call_submission.head()


new_df['k'] = new_df['k'].astype('int32')

new_df['k'].dtype

for i in range(len(new_df)):
    value = new_df['iv'].iloc[i]
    k = new_df['k'].iloc[i]
    col = f'put_iv_{k}'
    cols_to_match = [f'X{i}' for i in range(42)]  # Include 'underlying' if needed

    row_i = new_df.iloc[i]
    
    # Compare each row in test to row_i
    mask = (call_submission[cols_to_match] == row_i[cols_to_match]).all(axis=1)
    
    # Get matching index
    matching_index = call_submission.index[mask]

    # insert
    call_submission.loc[matching_index, col] = value


In [ ]:
call_submission.drop(columns = ['underlying']+[f'X{i}' for i in range(42)]).to_csv('/kaggle/working/submission4.csv',index = False)

In [ ]:
from IPython.display import FileLink
FileLink("submission4.parquet")

In [ ]:
torch.save(model.state_dict(), "trained_model.pth")
